In [ ]:
!pip install kagglehub

import kagglehub

cache_path = kagglehub.dataset_download(
    "sapnilpatel/tanishq-jewellery-dataset"
)

print("Downloaded to cache:", cache_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [kagglehub]


100%|████████████████████████████████████████████████████████████████████████████| 2.11M/2.11M [00:01<00:00, 1.37MB/s]

Extracting files...
Downloaded to cache: /home/akash/.cache/kagglehub/datasets/sapnilpatel/tanishq-jewellery-dataset/versions/1


In [ ]:
import os

PROJECT_ROOT = "/home/akash/Jewellary_RAG"
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "tanishq")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "embeddings"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "notebooks"), exist_ok=True)


In [ ]:
import shutil

for item in os.listdir(cache_path):
    src = os.path.join(cache_path, item)
    dst = os.path.join(DATA_DIR, item)

    if not os.path.exists(dst):
        shutil.move(src, dst)


In [ ]:
import os

BASE_DIR = "/home/akash/Jewellary_RAG/data/tanishq/Jewellery_Data"
print(os.listdir(BASE_DIR))


['necklace', 'ring']


In [ ]:
import os

UNIFIED_IMG_DIR = "/home/akash/Jewellary_RAG/data/tanishq/images"
os.makedirs(UNIFIED_IMG_DIR, exist_ok=True)


In [ ]:
import shutil
import json

BASE_DIR = "/home/akash/Jewellary_RAG/data/tanishq/Jewellery_Data"
image_metadata = []

for category in os.listdir(BASE_DIR):
    category_path = os.path.join(BASE_DIR, category)

    if not os.path.isdir(category_path):
        continue

    for fname in os.listdir(category_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            src = os.path.join(category_path, fname)

            # make filename unique (important!)
            new_name = f"{category}_{fname}"
            dst = os.path.join(UNIFIED_IMG_DIR, new_name)

            if not os.path.exists(dst):
                shutil.copy2(src, dst)

            image_metadata.append({
                "image_id": new_name,
                "category": category
            })


In [ ]:
META_PATH = "/home/akash/Jewellary_RAG/data/tanishq/image_metadata.json"

with open(META_PATH, "w") as f:
    json.dump(image_metadata, f, indent=2)

print("Saved metadata for", len(image_metadata), "images")


Saved metadata for 490 images


In [ ]:
print("Unified images:", len(os.listdir(UNIFIED_IMG_DIR)))
print(os.listdir(UNIFIED_IMG_DIR)[:5])

with open(META_PATH) as f:
    print(json.load(f)[:3])


Unified images: 490
['ring_ring_140.jpg', 'ring_ring_146.jpg', 'necklace_necklace_167.jpg', 'necklace_necklace_254.jpg', 'ring_ring_099.jpg']
[{'image_id': 'necklace_necklace_154.jpg', 'category': 'necklace'}, {'image_id': 'necklace_necklace_78.jpg', 'category': 'necklace'}, {'image_id': 'necklace_necklace_252.jpg', 'category': 'necklace'}]


In [ ]:
import os
import cv2
import json
from tqdm import tqdm

IMAGE_DIR = "/home/akash/Jewellary_RAG/data/tanishq/images"
OUTPUT_DIR = "/home/akash/Jewellary_RAG/data/tanishq"

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
import os
import torch
import clip
from PIL import Image
from tqdm import tqdm
import numpy as np


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
print("GPU:", torch.cuda.get_device_name(0))


Using device: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
model, preprocess = clip.load("ViT-B/16", device=device)

if device == "cuda":
    model = model.half()

model.eval()


100%|███████████████████████████████████████| 335M/335M [00:22<00:00, 15.6MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
import os
import torch
import clip
from PIL import Image
from tqdm import tqdm

DEVICE = "cuda"
IMAGE_DIR = "/home/akash/Jewellary_RAG/data/tanishq/images"
OUT_PATH = "/home/akash/Jewellary_RAG/embeddings/image_embeddings.pth"

model, preprocess = clip.load("ViT-B/16", device=DEVICE)

model.eval()

image_files = sorted([
    f for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

BATCH_SIZE = 16
all_embeddings = []
all_image_ids = []

with torch.no_grad():
    for i in tqdm(range(0, len(image_files), BATCH_SIZE)):
        batch = image_files[i:i+BATCH_SIZE]
        images, ids = [], []

        for fname in batch:
            try:
                img = Image.open(os.path.join(IMAGE_DIR, fname)).convert("RGB")
                images.append(preprocess(img))
                ids.append(fname)
            except:
                continue

        if not images:
            continue

        image_tensor = torch.stack(images).to(DEVICE).half()
        emb = model.encode_image(image_tensor)
        emb = emb / emb.norm(dim=-1, keepdim=True)

        all_embeddings.append(emb.cpu())
        all_image_ids.extend(ids)

embeddings = torch.cat(all_embeddings)

torch.save({
    "image_ids": all_image_ids,
    "embeddings": embeddings,
    "model": "CLIP ViT-B/16",
    "normalized": True
}, OUT_PATH)

print("✅ Saved CLIP embeddings:", embeddings.shape)


100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:02<00:00, 13.71it/s]

✅ Saved CLIP embeddings: torch.Size([490, 512])


captions.json	     images	     rejected_images.json
image_metadata.json  Jewellery_Data  valid_images.json


In [ ]:
import os
import json
import torch
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

IMAGE_DIR = "/home/akash/Jewellary_RAG/data/tanishq/images"
OUT_PATH = "/home/akash/Jewellary_RAG/data/tanishq/blip_captions.json"
DEVICE = "cuda"

processor = BlipProcessor.from_pretrained(
    "Salesforce/blip-image-captioning-base",
    use_fast=True
)
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(DEVICE)

model.eval()

captions = {}

for fname in tqdm(sorted(os.listdir(IMAGE_DIR))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    image = Image.open(os.path.join(IMAGE_DIR, fname)).convert("RGB")

    inputs = processor(
        image,
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=25,
            num_beams=3
        )

    caption = processor.decode(output[0], skip_special_tokens=True)
    captions[fname] = caption

with open(OUT_PATH, "w") as f:
    json.dump(captions, f, indent=2)

print("✅ New BLIP captions saved:", len(captions))
print("📄 File:", OUT_PATH)


100%|███████████████████████████████████████████████████████████████████████████████| 490/490 [00:52<00:00,  9.27it/s]

✅ New BLIP captions saved: 490
📄 File: /home/akash/Jewellary_RAG/data/tanishq/blip_captions.json


In [ ]:
import json
import random

with open(OUT_PATH, "r") as f:
    data = json.load(f)

print("Total captions:", len(data))

for img, cap in random.sample(list(data.items()), 5):
    print(f"\n🖼️ {img}")
    print(f"📝 {cap}")


Total captions: 490

🖼️ necklace_necklace_15.jpg
📝 a gold necklace with green stones on a white background

🖼️ necklace_necklace_164.jpg
📝 a gold plated necklace with a triangle pendant

🖼️ ring_ring_031.jpg
📝 a white gold ring with diamonds

🖼️ ring_ring_063.jpg
📝 a diamond and white gold ring

🖼️ ring_ring_146.jpg
📝 a yellow gold ring with three diamonds


In [ ]:
rm /home/akash/Jewellary_RAG/data/tanishq/blip_captions.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import json
import re

def extract_json(text: str) -> dict:
    """
    Safely extract JSON object from LLM output.
    """
    text = text.strip()

    # Try direct parse first
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # Try to extract JSON block
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            pass

    raise ValueError(f"Invalid JSON output:\n{text}")


In [ ]:
SYSTEM_PROMPT = """
You are a strict metadata extraction system.

Your task is to classify jewelry metadata using ONLY the provided caption.
If a value cannot be determined directly from the caption, return "unknown".

STRICT RULES:
- Output VALID JSON only
- Use ONLY the allowed values listed below
- Do NOT invent, infer, or guess
- Do NOT repeat the caption
- Use lowercase exactly
- If unsure, choose "unknown"

ALLOWED VALUES:

category:
- necklace
- ring
- unknown

metal:
- yellow gold
- white gold
- rose gold
- gold
- silver
- gold plated
- unknown

primary_stone:
- diamond
- pearl
- sapphire
- emerald
- ruby
- topaz
- colored stone
- unknown

form:
- chain
- pendant
- band
- engagement
- wedding
- eternity
- unknown

CONFIDENCE RULES:
- All confidence values between 0.0 and 1.0
- category <= 0.95
- metal <= 0.6
- primary_stone <= 0.6
- form <= 0.6
- If value is "unknown", confidence MUST be <= 0.4

SPECIAL DATASET RULE:
- The dataset contains ONLY necklaces and rings.
- If the caption contains the word "bracelet", treat it as "necklace"
  (this is caption noise from the vision model, not a real category).


OUTPUT FORMAT:

{
  "category": "",
  "metal": "",
  "primary_stone": "",
  "form": "",
  "confidence": {
    "category": 0.0,
    "metal": 0.0,
    "primary_stone": 0.0,
    "form": 0.0
  }
}
"""


In [ ]:
from openai import OpenAI
client=OpenAI(api_key="sk-1mNkjU09WUAejXBxWhaA_w",base_url="https://apidev.navigatelabsai.com/")
def extract_metadata_labels(caption: str) -> dict:
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0.0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "user",
                "content": f"""
Caption:
"{caption}"

Return JSON:
{{
  "category": "",
  "metal": "",
  "primary_stone": "",
  "style": "",
  "confidence": {{
    "category": 0.0,
    "metal": 0.0,
    "primary_stone": 0.0,
    "style": 0.0
  }}
}}
"""
            }
        ]
    )

    return extract_json(response.choices[0].message.content)


In [ ]:
def normalize_category(category: str, caption: str) -> str:
    caption_l = caption.lower()

    if "ring" in caption_l:
        return "ring"

    if "necklace" in caption_l or "bracelet" in caption_l:
        return "necklace"

    return "unknown"


In [ ]:
def normalize_form(form: str, caption: str) -> str:
    if "chain" in caption:
        return "chain"
    return "unknown"


In [ ]:
def metal_from_caption(caption: str) -> str | None:
    c = caption.lower()

    # ordered from most specific → least specific
    METAL_RULES = [
        ("rose gold", "rose gold"),
        ("white gold", "white gold"),
        ("yellow gold", "yellow gold"),
        ("gold plated", "gold plated"),
        ("plated gold", "gold plated"),
        ("sterling silver", "silver"),
        ("silver plated", "silver"),
        ("silver", "silver"),
        ("18k gold", "gold"),
        ("14k gold", "gold"),
        ("solid gold", "gold"),
        ("gold tone", "gold"),
        ("gold-colored", "gold"),
        ("gold", "gold"),
    ]

    for phrase, metal in METAL_RULES:
        if phrase in c:
            return metal

    return None


In [ ]:
def normalize_metadata(m: dict) -> dict:
    # defensive copy
    m = dict(m)
    m["confidence"] = dict(m.get("confidence", {}))

    caption = m.get("_caption", "").lower()

    # --- normalize metal ---
    METAL_MAP = {
        "gold": "gold",
        "yellow gold": "yellow gold",
        "white gold": "white gold",
        "rose gold": "rose gold",
        "silver": "silver",
        "gold plated": "gold plated",
    }

    # --- normalize primary stone ---
    STONE_MAP = {
        "diamond": "diamond",
        "pearl": "pearl",
        "sapphire": "sapphire",
        "emerald": "emerald",
        "ruby": "ruby",
        "topaz": "topaz",
        "colored stone": "colored stone",
    }
    m["primary_stone"] = STONE_MAP.get(m.get("primary_stone"), "unknown")

    # --- normalize category (bracelet → necklace) ---
    # =====================================================
    # 🔒 HARD DATASET GUARD (dataset has only rings/necklaces)
    # =====================================================
    if m["category"] == "unknown":
        m["category"] = "necklace"
        m["confidence"]["category"] = min(
            float(m["confidence"].get("category", 0.4)),
            0.4
        )


    # --- normalize form (explicit only) ---
    m["form"] = normalize_form(
        m.get("form", "unknown"),
        caption
    )

    # =====================================================
    # 🔹 CONTROLLED GOLD FALLBACK (ADD HERE)
    # =====================================================
    # --- normalize metal ---
    m["metal"] = METAL_MAP.get(m.get("metal"), "unknown")

    # 🔹 CONTROLLED caption-based metal fix (generalized)
    if m["metal"] == "unknown":
        caption_metal = metal_from_caption(caption)
        if caption_metal:
            m["metal"] = caption_metal
            m["confidence"]["metal"] = min(
                float(m["confidence"].get("metal", 0.4)),
                0.4
            )


    # --- confidence caps ---
    caps = {
        "category": 0.95,
        "metal": 0.6,
        "primary_stone": 0.6,
        "form": 0.6
    }

    for k, cap in caps.items():
        m["confidence"][k] = min(float(m["confidence"].get(k, 0.0)), cap)
        if m.get(k) == "unknown":
            m["confidence"][k] = min(m["confidence"][k], 0.4)

    # cleanup temporary field
    m.pop("_caption", None)

    return m

In [ ]:
def build_metadata_text(m: dict) -> str:
    parts = []

    if m["category"] != "unknown":
        parts.append(m["category"])

    if m["metal"] != "unknown":
        parts.append(f"made of {m['metal']}")

    if m["primary_stone"] != "unknown":
        parts.append(f"with {m['primary_stone']}")

    if m["form"] != "unknown":
        parts.append(f"{m['form']} type")

    return " ".join(parts) if parts else "jewelry"


In [ ]:
def get_final_metadata(caption: str) -> dict:
    raw = extract_metadata_labels(caption)

    if not isinstance(raw, dict):
        raise ValueError("extract_metadata_labels did not return a dict")

    normalized = normalize_metadata(raw)

    # ensure required keys exist
    for key in ["category", "metal", "primary_stone", "style", "confidence"]:
        if key not in normalized:
            raise ValueError(f"Missing key in metadata: {key}")

    normalized["metadata_text"] = build_metadata_text(normalized)

    return normalized


In [ ]:
import json

CAPTIONS_PATH = "/home/akash/Jewellary_RAG/data/tanishq/blip_captions.json"

with open(CAPTIONS_PATH, "r") as f:
    captions = json.load(f)

print("Total captions:", len(captions))


Total captions: 490


In [ ]:
def extract_metadata_batch(items):
    """
    items: list of (image_id, caption)
    """

    user_prompt = """
You will receive multiple captions.

For EACH caption:
- Extract metadata strictly following ALL system rules
- Use ONLY allowed values
- Output VALID JSON ONLY

Return a JSON ARRAY.
Each array element MUST strictly match the system OUTPUT FORMAT.
Add an extra field "image_id" to each object.

Do not include any extra text.
"""

    for image_id, caption in items:
        user_prompt += f"""
Caption:
"{caption}"
Image ID: {image_id}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        temperature=0.0,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},  # ✅ unchanged
            {"role": "user", "content": user_prompt},
        ]
    )

    return extract_json(response.choices[0].message.content)


In [ ]:
test_batch = list(captions.items())[:10]


In [ ]:
print("\n🔍 Batch-1 sanity check (after fixes):\n")

for (image_id, caption), md in zip(test_batch, processed):
    print(f"Image ID        : {image_id}")
    print(f"Caption         : {caption}")
    print(f"Category        : {md['category']}")
    print(f"Metal           : {md['metal']}")
    print(f"Primary Stone   : {md['primary_stone']}")
    print(f"Form            : {md['form']}")
    print(f"Metadata Text   : {md['metadata_text']}")
    print(f"Confidence      : {md['confidence']}")
    print("-" * 70)



🔍 Batch-1 sanity check (after fixes):

Image ID        : necklace_necklace_1.jpg
Caption         : a gold necklace with white pearls on a white background
Category        : necklace
Metal           : gold
Primary Stone   : pearl
Form            : unknown
Metadata Text   : necklace made of gold with pearl
Confidence      : {'category': 0.95, 'metal': 0.4, 'primary_stone': 0.6, 'form': 0.4}
----------------------------------------------------------------------
Image ID        : necklace_necklace_10.jpg
Caption         : a gold bracelet with a knot design on it
Category        : necklace
Metal           : gold
Primary Stone   : unknown
Form            : unknown
Metadata Text   : necklace made of gold
Confidence      : {'category': 0.95, 'metal': 0.4, 'primary_stone': 0.4, 'form': 0.4}
----------------------------------------------------------------------
Image ID        : necklace_necklace_100.jpg
Caption         : a gold necklace with a pearl on it
Category        : necklace
Metal      

In [ ]:
results = extract_metadata_batch(test_batch)


In [ ]:
processed = []

caption_lookup = dict(test_batch)

for r in results:
    r["_caption"] = caption_lookup.get(r["image_id"], "")
    md = normalize_metadata(r)
    md["metadata_text"] = build_metadata_text(md)
    processed.append(md)


In [ ]:
from tqdm import tqdm

def chunked(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i:i + size]

all_metadata = {}
failed = []

items = list(captions.items())
caption_lookup = dict(items)

for batch in tqdm(chunked(items, 10), total=(len(items) + 9) // 10):
    try:
        results = extract_metadata_batch(batch)

        for r in results:
            # 🔹 attach caption for normalization
            r["_caption"] = caption_lookup.get(r["image_id"], "")

            md = normalize_metadata(r)
            md["metadata_text"] = build_metadata_text(md)

            all_metadata[r["image_id"]] = md

    except Exception as e:
        failed.append({
            "batch_image_ids": [img_id for img_id, _ in batch],
            "error": str(e)
        })


100%|████████████████████████████████████████████████████████████████████████████| 49/49 [09:30<00:00, 11.64s/it]


In [ ]:
import json

output_path = "jewellery_metadata_v1.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Metadata saved to {output_path}")
print(f"Total items: {len(all_metadata)}")
print(f"Failed batches: {len(failed)}")


✅ Metadata saved to jewellery_metadata_v1.json
Total items: 485
Failed batches: 0


In [ ]:
print("Captions:", len(captions))
print("Metadata:", len(all_metadata))


Captions: 490
Metadata: 488


In [ ]:
caption_ids = set(captions.keys())
metadata_ids = set(all_metadata.keys())

missing_ids = caption_ids - metadata_ids

print("Captions:", len(caption_ids))
print("Metadata:", len(metadata_ids))
print("Missing metadata count:", len(missing_ids))

print("\nMissing image IDs:")
for img in sorted(missing_ids):
    print(img)


Captions: 490
Metadata: 488
Missing metadata count: 2

Missing image IDs:
ring_ring_008.jpg
ring_ring_142.jpg


In [ ]:
# the two known missing images
missing_ids = {"ring_ring_008.jpg", "ring_ring_142.jpg"}

# rebuild batch input in the same format
retry_batch = [(img_id, captions[img_id]) for img_id in missing_ids]

try:
    results = extract_metadata_batch(retry_batch)

    result_ids = {r["image_id"] for r in results}
    still_missing = missing_ids - result_ids

    for r in results:
        r["_caption"] = captions.get(r["image_id"], "")
        md = normalize_metadata(r)
        md["metadata_text"] = build_metadata_text(md)
        all_metadata[r["image_id"]] = md
        print(f"✅ Fixed metadata for {r['image_id']}")

    if still_missing:
        print("❌ Still missing after retry:", still_missing)

except Exception as e:
    print("❌ Retry batch failed:", e)


✅ Fixed metadata for ring_ring_008.jpg
✅ Fixed metadata for ring_ring_142.jpg


In [ ]:
import json

with open("jewellery_metadata_v1.json", "w", encoding="utf-8") as f:
    json.dump(all_metadata, f, indent=2, ensure_ascii=False)

assert len(all_metadata) == 490
print("✅ Metadata complete: 490 / 490")


✅ Metadata complete: 490 / 490


In [ ]:
def build_canonical_metadata_text(m: dict) -> str:

    return (
        f"{m['category']} jewelry "
        f"material {m['metal']} "
        f"stone {m['primary_stone']} "
        f"form {m['form']}"
    )


In [ ]:
from collections import Counter

canonical_texts = [
    build_canonical_metadata_text(m)
    for m in all_metadata.values()
]

counter = Counter(canonical_texts)

print("\nTop 10 most common canonical texts:\n")
for text, count in counter.most_common(10):
    print(f"{count:>3}  →  {text}")



Top 10 most common canonical texts:

 76  →  necklace jewelry material gold stone unknown form unknown
 62  →  ring jewelry material yellow gold stone diamond form unknown
 55  →  necklace jewelry material gold stone unknown form chain
 40  →  ring jewelry material white gold stone diamond form unknown
 31  →  necklace jewelry material gold stone diamond form unknown
 30  →  necklace jewelry material unknown stone unknown form unknown
 22  →  necklace jewelry material yellow gold stone unknown form chain
 13  →  necklace jewelry material gold stone pearl form unknown
 13  →  ring jewelry material yellow gold stone unknown form unknown
 12  →  necklace jewelry material yellow gold stone unknown form unknown


In [ ]:
!pip install chromadb


In [ ]:
import chromadb
from chromadb.config import Settings
import torch
import clip
import json
import numpy as np


In [ ]:
import torch
import json
import numpy as np

PROJECT_ROOT = "/home/akash/Jewellary_RAG"

IMAGE_EMB_PATH = "/home/akash/Jewellary_RAG/embeddings/clip_image_embeddings_vit_b16.pth"
METADATA_JSON  = "/home/akash/jewellery_metadata_v1.json"

image_data = torch.load(IMAGE_EMB_PATH)

image_ids        = image_data["image_ids"]
image_embeddings = image_data["embeddings"].numpy()

print("✅ Image embeddings loaded")
print("Images:", len(image_ids))
print("Embedding shape:", image_embeddings.shape)


✅ Image embeddings loaded
Images: 490
Embedding shape: (490, 512)


In [ ]:
with open(METADATA_JSON, "r", encoding="utf-8") as f:
    all_metadata = json.load(f)

print("✅ Metadata loaded")
print("Metadata items:", len(all_metadata))


✅ Metadata loaded
Metadata items: 490


In [ ]:
import json, os, tempfile

FINAL_METADATA_PATH = "/home/akash/Jewellary_RAG/data/tanishq/jewellery_metadata_v1.json"

assert isinstance(all_metadata, dict)
assert len(all_metadata) == 490, "Metadata incomplete — refusing to write"

tmp_path = FINAL_METADATA_PATH + ".tmp"

with open(tmp_path, "w", encoding="utf-8") as f:
    json.dump(all_metadata, f, indent=2, ensure_ascii=False)

os.replace(tmp_path, FINAL_METADATA_PATH)

print("✅ Metadata safely persisted at:", FINAL_METADATA_PATH)


✅ Metadata safely persisted at: /home/akash/Jewellary_RAG/data/tanishq/jewellery_metadata_v1.json


In [ ]:
canonical_texts = [
    build_canonical_metadata_text(all_metadata[img_id])
    for img_id in image_ids
]

print("✅ Canonical metadata text ready")


✅ Canonical metadata text ready


In [ ]:
for i in range(3):
    print(image_ids[i], "→", canonical_texts[i])


necklace_necklace_1.jpg → necklace jewelry material gold stone pearl form unknown
necklace_necklace_10.jpg → necklace jewelry material gold stone unknown form unknown
necklace_necklace_100.jpg → necklace jewelry material gold stone pearl form unknown


In [ ]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model, _ = clip.load("ViT-B/16", device=device)
clip_model.eval()

# Removed: if device == "cuda":
# Removed:     clip_model = clip_model.half() # This was causing the RuntimeError

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
def encode_texts_clip(texts, batch_size=32):
    embs = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            tokens = clip.tokenize(batch).to(device)

            e = clip_model.encode_text(tokens)
            e = e / e.norm(dim=-1, keepdim=True)

            embs.append(e.cpu().numpy())

    return np.vstack(embs)


In [ ]:
metadata_embeddings = encode_texts_clip(canonical_texts)

assert metadata_embeddings.shape[0] == 490
print("✅ Metadata text embeddings:", metadata_embeddings.shape)

✅ Metadata text embeddings: (490, 512)


In [ ]:
# chroma_client = chromadb.Client(
#     Settings(
#         persist_directory="/home/akash/Jewellary_RAG/chroma",
#         anonymized_telemetry=False
#     )
# )

In [ ]:
chroma_client = chromadb.Client(
    Settings(
        persist_directory="/home/akash/Jewellary_RAG/chroma",
        anonymized_telemetry=False
    )
)
image_collection = chroma_client.get_or_create_collection(
    name="jewelry_images",
    metadata={"hnsw:space": "cosine"}
)


In [ ]:
image_collection.add(
    ids=image_ids,
    embeddings=image_embeddings.tolist(),
    metadatas=[
        {
            "image_id": img_id,
            "category": all_metadata[img_id]["category"]
        }
        for img_id in image_ids
    ]
)

print("✅ Image collection indexed")


✅ Image collection indexed


In [ ]:
metadata_collection = chroma_client.get_or_create_collection(
    name="jewelry_metadata",
    metadata={"hnsw:space": "cosine"}
)


In [ ]:
def flatten_metadata_for_chroma(img_id):
    m = all_metadata[img_id]
    c = m["confidence"]

    return {
        "image_id": img_id,
        "category": m["category"],
        "metal": m["metal"],
        "primary_stone": m["primary_stone"],
        "form": m["form"],

        # flattened confidence
        "conf_category": float(c.get("category", 0.0)),
        "conf_metal": float(c.get("metal", 0.0)),
        "conf_primary_stone": float(c.get("primary_stone", 0.0)),
        "conf_form": float(c.get("form", 0.0)),
    }


In [ ]:
metadata_collection.add(
    ids=image_ids,
    embeddings=metadata_embeddings.tolist(),
    documents=canonical_texts,
    metadatas=[
        flatten_metadata_for_chroma(img_id)
        for img_id in image_ids
    ]
)

print("✅ Metadata collection indexed (flattened)")


✅ Metadata collection indexed (flattened)


In [ ]:
  collections = chroma_client.list_collections()
  print(collections)


[Collection(name=jewelry_metadata), Collection(name=jewelry_images)]


In [ ]:
image_collection = chroma_client.get_collection("jewelry_images")
metadata_collection = chroma_client.get_collection("jewelry_metadata")

print("Image collection count   :", image_collection.count())
print("Metadata collection count:", metadata_collection.count())


Image collection count   : 490
Metadata collection count: 490


In [ ]:
[chroma.name for chroma in chroma_client.list_collections()]


['jewelry_metadata', 'jewelry_images']

In [ ]:
print(chroma_client.get_collection("jewelry_images").count())
print(chroma_client.get_collection("jewelry_metadata").count())


490
490


In [ ]:
image_collection = chroma_client.get_collection("jewelry_images")

img_sample = image_collection.peek(5)

print("📸 IMAGE COLLECTION SAMPLE\n")

for i in range(len(img_sample["ids"])):
    print(f"#{i+1}")
    print("ID       :", img_sample["ids"][i])
    print("Metadata :", img_sample["metadatas"][i])
    print("Embedding dim:", len(img_sample["embeddings"][i]))
    print("-" * 60)


📸 IMAGE COLLECTION SAMPLE

#1
ID       : necklace_necklace_1.jpg
Metadata : {'image_id': 'necklace_necklace_1.jpg', 'category': 'necklace'}
Embedding dim: 512
------------------------------------------------------------
#2
ID       : necklace_necklace_10.jpg
Metadata : {'category': 'necklace', 'image_id': 'necklace_necklace_10.jpg'}
Embedding dim: 512
------------------------------------------------------------
#3
ID       : necklace_necklace_100.jpg
Metadata : {'image_id': 'necklace_necklace_100.jpg', 'category': 'necklace'}
Embedding dim: 512
------------------------------------------------------------
#4
ID       : necklace_necklace_101.jpg
Metadata : {'image_id': 'necklace_necklace_101.jpg', 'category': 'necklace'}
Embedding dim: 512
------------------------------------------------------------
#5
ID       : necklace_necklace_102.jpg
Metadata : {'category': 'necklace', 'image_id': 'necklace_necklace_102.jpg'}
Embedding dim: 512
-------------------------------------------------------

In [ ]:
metadata_collection = chroma_client.get_collection("jewelry_metadata")

meta_sample = metadata_collection.peek(5)

print("🏷️ METADATA COLLECTION SAMPLE\n")

for i in range(len(meta_sample["ids"])):
    print(f"#{i+1}")
    print("ID       :", meta_sample["ids"][i])
    print("Document :", meta_sample["documents"][i])
    print("Metadata :")
    for k, v in meta_sample["metadatas"][i].items():
        print(f"  - {k}: {v}")
    print("Embedding dim:", len(meta_sample["embeddings"][i]))
    print("-" * 60)


🏷️ METADATA COLLECTION SAMPLE

#1
ID       : necklace_necklace_1.jpg
Document : necklace jewelry material gold stone pearl form unknown
Metadata :
  - conf_primary_stone: 0.6
  - conf_metal: 0.4
  - primary_stone: pearl
  - category: necklace
  - conf_form: 0.4
  - conf_category: 0.95
  - metal: gold
  - form: unknown
  - image_id: necklace_necklace_1.jpg
Embedding dim: 512
------------------------------------------------------------
#2
ID       : necklace_necklace_10.jpg
Document : necklace jewelry material gold stone unknown form unknown
Metadata :
  - form: unknown
  - image_id: necklace_necklace_10.jpg
  - primary_stone: unknown
  - conf_primary_stone: 0.4
  - metal: gold
  - conf_category: 0.95
  - conf_form: 0.4
  - conf_metal: 0.4
  - category: necklace
Embedding dim: 512
------------------------------------------------------------
#3
ID       : necklace_necklace_100.jpg
Document : necklace jewelry material gold stone pearl form unknown
Metadata :
  - image_id: necklace_necklace

In [ ]:
ids_img  = image_collection.peek(10)["ids"]
ids_meta = metadata_collection.peek(10)["ids"]

print("🔗 ID ALIGNMENT CHECK\n")

for i in range(10):
    print(f"{i+1}. image_id:", ids_img[i], "| metadata_id:", ids_meta[i])


🔗 ID ALIGNMENT CHECK

1. image_id: necklace_necklace_1.jpg | metadata_id: necklace_necklace_1.jpg
2. image_id: necklace_necklace_10.jpg | metadata_id: necklace_necklace_10.jpg
3. image_id: necklace_necklace_100.jpg | metadata_id: necklace_necklace_100.jpg
4. image_id: necklace_necklace_101.jpg | metadata_id: necklace_necklace_101.jpg
5. image_id: necklace_necklace_102.jpg | metadata_id: necklace_necklace_102.jpg
6. image_id: necklace_necklace_103.jpg | metadata_id: necklace_necklace_103.jpg
7. image_id: necklace_necklace_104.jpg | metadata_id: necklace_necklace_104.jpg
8. image_id: necklace_necklace_105.jpg | metadata_id: necklace_necklace_105.jpg
9. image_id: necklace_necklace_106.jpg | metadata_id: necklace_necklace_106.jpg
10. image_id: necklace_necklace_107.jpg | metadata_id: necklace_necklace_107.jpg


In [ ]:
import clip, torch, numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

clip_model, _ = clip.load("ViT-B/16", device=device)
clip_model = clip_model.float()
clip_model.eval()

query = "gold necklace with pearl"

tokens = clip.tokenize([query]).to(device)
with torch.no_grad():
    q_emb = clip_model.encode_text(tokens)
    q_emb = q_emb / q_emb.norm(dim=-1, keepdim=True)

res = metadata_collection.query(
    query_embeddings=q_emb.cpu().numpy().tolist(),
    n_results=5
)

print("\n🔎 METADATA QUERY RESULT\n")

for i, img_id in enumerate(res["ids"][0]):
    print(f"{i+1}. {img_id}")
    print("   →", res["documents"][0][i])



🔎 METADATA QUERY RESULT

1. necklace_necklace_1.jpg
   → necklace jewelry material gold stone pearl form unknown
2. necklace_necklace_114.jpg
   → necklace jewelry material gold stone pearl form unknown
3. necklace_necklace_154.jpg
   → necklace jewelry material gold stone pearl form unknown
4. necklace_necklace_85.jpg
   → necklace jewelry material gold stone pearl form unknown
5. necklace_necklace_86.jpg
   → necklace jewelry material gold stone pearl form unknown


In [ ]:
print(
    image_collection.count(),
    metadata_collection.count()
)
# expect: 490 490


490 490


In [ ]:
client.persist()
print("✅ Chroma DB persisted to disk")


NameError: name 'client' is not defined

In [ ]:
ls -lh /home/akash/Jewellary_RAG/chroma


total 4.0K
-rw-rw-r-- 1 akash akash 36 Feb  4 15:02 telemetry_user_id


In [ ]:
rm -rf /home/akash/Jewellary_RAG/chroma


In [ ]:
!find /home/akash -name "chroma.sqlite3" 2>/dev/null

/home/akash/Resume_Score/backend/chroma_db/chroma.sqlite3


In [ ]:
!find /home/akash -type d -name "collections" 2>/dev/null | grep chroma


In [ ]:
!find /home/akash -type d -name "index" 2>/dev/null | grep chroma


In [ ]:
!mkdir -p /home/akash/Jewellary_RAG/chroma
!cp -r /home/akash/Resume_Score/backend/chroma_db \
      /home/akash/Jewellary_RAG/chroma/


In [ ]:
ls /home/akash/Jewellary_RAG/chroma/chroma_db


a75ad345-3d55-499f-a646-e9f88633345b/  e641946b-0770-4ec2-bc5f-a296f7517412/
chroma.sqlite3


In [ ]:
# Force a no-op write that triggers persistence
test_collection = chroma_client.get_collection("jewelry_images")

test_collection.add(
    ids=["__persist_probe__"],
    embeddings=[[0.0] * 512],
    metadatas=[{"probe": True}]
)

test_collection.delete(ids=["__persist_probe__"])


In [ ]:
# internal, read-only inspection
system = chroma_client._system
print(system)


In [ ]:
print(system.settings)


environment='' chroma_api_impl='chromadb.api.rust.RustBindingsAPI' chroma_server_nofile=None chroma_server_thread_pool_size=40 tenant_id='default' topic_namespace='default' chroma_server_host=None chroma_server_headers=None chroma_server_http_port=None chroma_server_ssl_enabled=False chroma_server_ssl_verify=None chroma_server_api_default_path=<APIVersion.V2: '/api/v2'> chroma_server_cors_allow_origins=[] chroma_http_keepalive_secs=40.0 chroma_http_max_connections=None chroma_http_max_keepalive_connections=None is_persistent=False persist_directory='/home/akash/Jewellary_RAG/chroma' chroma_memory_limit_bytes=0 chroma_segment_cache_policy=None allow_reset=False chroma_auth_token_transport_header=None chroma_client_auth_provider=None chroma_client_auth_credentials=None chroma_server_auth_ignore_paths={'/api/v2': ['GET'], '/api/v2/heartbeat': ['GET'], '/api/v2/version': ['GET'], '/api/v1': ['GET'], '/api/v1/heartbeat': ['GET'], '/api/v1/version': ['GET']} chroma_overwrite_singleton_tenant